<a href="https://colab.research.google.com/github/Arvin-007Elite/PYTHON/blob/master/NLP_Twitter_Sentiment_An_alysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install kaggle

In [9]:
# Create the .kaggle directory in the home directory if it doesn't exist
!mkdir -p ~/.kaggle

# Copy the kaggle.json file to the .kaggle directory
!cp kaggle.json ~/.kaggle/

# Set the permissions of the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json


In [11]:
# Api to fetch dataset from kaggle
# go  to  profile of kaggle , cpy api cmnd !
!kaggle datasets download  -d kazanova/sentiment140
# instead of uploading from system , we r getting it from api

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 82% 66.0M/80.9M [00:00<00:00, 185MB/s]
100% 80.9M/80.9M [00:00<00:00, 181MB/s]


In [14]:
# Extracting the zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

# now open and extract
# r means read

with ZipFile(dataset , 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [23]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# data preprocessing
# Read the CSV file with error handling

import csv
import pandas as pd

file_path = '/content/training.1600000.processed.noemoticon.csv'
rows = []

try:
    with open(file_path, encoding='ISO-8859-1') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            rows.append(row)
    twitter_data = pd.DataFrame(rows[1:], columns=rows[0])
    print("File loaded successfully using csv library.")
except csv.Error as e:
    print(f"csv.Error: {e}")



# use this code , not normal read csv , cause datset has unwanted commas in it







File loaded successfully using csv library.


In [69]:
twitter_data.shape

(5795768, 6)

In [70]:
twitter_data.head()

,target,id,date,flag,user,text
0,0.0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0.0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0.0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0.0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0.0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [49]:
# above datset took the 1 st column as headers , so we have to tell that
# will name the columns and read dataest again

# will name it according the col names mentioned in kaggle
column_names=['target' , 'id', 'date' , 'flag' , 'user' , 'text']

# agin will make it read



try:
    with open(file_path, encoding='ISO-8859-1') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            rows.append(row)
    twitter_data = pd.DataFrame(rows[1:], columns=column_names)
    print("File loaded successfully using csv library.")
except csv.Error as e:
    print(f"csv.Error: {e}")

File loaded successfully using csv library.


In [50]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [51]:
twitter_data.shape

(5795768, 6)

In [52]:
# column names changed
# 0 is negative  , 1 is positive

twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      1
dtype: int64

In [53]:
twitter_data['target'].value_counts()

target
4            3200000
0            2595767
98863667"          1
Name: count, dtype: int64

In [45]:
# import csv
# import pandas as pd

# file_path = '/content/training.1600000.processed.noemoticon.csv'
# column_names = ['target', 'id', 'date', 'flag', 'user', 'text']  # Replace with actual column names
# rows = []

# expected_columns = len(column_names)

# try:
#     with open(file_path, encoding='ISO-8859-1') as csvfile:
#         reader = csv.reader(csvfile)
#         for row in reader:
#             # Check if the row has the expected number of columns
#             if len(row) == expected_columns:
#                 rows.append(row)
#             else:
#                 print(f"Skipping malformed row: {row}")
#     twitter_data = pd.DataFrame(rows[1:], columns=column_names)  # Use provided column names
#     print("File loaded successfully using csv library.")
# except csv.Error as e:
#     print(f"csv.Error: {e}")

# # Display the first few rows of the DataFrame to verify
# print(twitter_data.head())


Skipping malformed row: ['98863667"', 'Mon Jun 01 19:14:57 PDT 2009', 'NO_QUERY', 'ana_cassini', "Unf...Mel didn't twit today!  Watchin' some TV (MTV),taking a look at the forum,listening to some music! Yeay,I'm a multiple tasks person!"]
File loaded successfully using csv library.
  target          id                          date      flag           user  \
0      0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY  scotthamilton   
1      0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY       mattycus   
2      0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY        ElleCTF   
3      0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY         Karoli   
4      0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY       joy_wolf   

                                                text  
0  is upset that he can't update his Facebook by ...  
1  @Kenichan I dived many times for the ball. Man...  
2    my whole body feels itchy and like its on fire   
3  @nationwideclass no

In [56]:
twitter_data['target'].value_counts()

target
4            3200000
0            2595767
98863667"          1
Name: count, dtype: int64

In [58]:
# converting label name 4 to 1

print(twitter_data['target'].dtype)



object


In [59]:
twitter_data['target'] = pd.to_numeric(twitter_data['target'], errors='coerce')

In [60]:
print(twitter_data['target'].unique())


[ 0.  4. nan]


In [61]:
# Replace 4 with 1 in the 'target' column
twitter_data.replace({'target': {4: 1}}, inplace=True)

In [62]:
# Verify the replacement
print("Unique values in 'target' column after replacement:", twitter_data['target'].unique())

Unique values in 'target' column after replacement: [ 0.  1. nan]


In [63]:
# Stemming
port_stem = PorterStemmer()

In [66]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]' , ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [72]:
twiiter_data.head()

NameError: name 'twiiter_data' is not defined